# Explore if Analytical Solution Works Better than Numerical

In [ ]:
import casadi as cas
import numpy as np
import matplotlib.pyplot as plt

from cas_models.discrete_time.models import StateSpaceModelDT

from feed_conc_ctrl.models import MixingTankModelDT
from cas_models.discrete_time.simulate import make_n_step_simulation_function_from_model

In [ ]:
def analytical_solution_general(t, L0, m0, v_dot_in, conc_in, v_dot_out, A):
    """Analytical solution to the initial value problem

        dL/dt = (v_dot_in - v_dot_out) / A
        dm/dt = v_dot_in * conc_in - v_dot_out * m / (A * L)

    This is the general analytical solution when v_dot_in != v_dot_out.
    """
    V0 = A * L0
    V_t = V0 + (v_dot_in - v_dot_out) * t

    # Exponent in the integrating factor
    exponent = v_dot_out / (v_dot_in - v_dot_out)

    # Integrating factor: (V0 + (v_dot_in - v_dot_out)*t)^exponent
    L_t = L0 + (v_dot_in - v_dot_out) * t / A
    m_t = (
        conc_in * V_t
        + (V0**exponent * m0 - conc_in * V0**(v_dot_in / (v_dot_in - v_dot_out)))
        * V_t**(-exponent)
    )

    return L_t, m_t


def analytical_solution_equal(t, L0, m0, v, conc_in, A):
    """Analytical solution for the case of equal flows (v_dot_in = 
    v_dot_out = v).
    """
    L_t = L0  # Constant level
    V0 = A * L0
    m_equilibrium = V0 * conc_in
    m_t = m_equilibrium + (m0 - m_equilibrium) * np.exp(-v * t / V0)
    return L_t, m_t

In [ ]:
def ivp_solution(t, x1_init, x2_init, u1, u2, u3, A, exp=np.exp, log=np.log):
    """Analytical solution to the initial value problem

        dL/dt = (v_dot_in - v_dot_out) / A
        dm/dt = v_dot_in * conc_in - v_dot_out * m / (A * L)

    System states: 
    x1 : Tank level, L [m]
    x2 : Total mass of suspended mineral in tank, m [tons]

    Inputs
    u1 : volumetric flowrate into tank, v_dot_in [m^3/hr]
    u2 : density of fluid entering tank, rho_in [tons/m^3]
    u3 : volumetric flowrate out of tank, v_dot_out [m^3/hr]
    """

    x1_final = x1_init + t * (u1 - u3) / A
    x2_final = (
        u2 * (A * x1_init + t * u1 - t * u3)
        + (
            (A * x1_init) ** (u3 / (u1 - u3)) * x2_init
            - u2 * exp(u1 * log(A * x1_init) / (u1 - u3))
        ) * exp(-u3 * log(A * x1_init + t * u1 - t * u3) / (u1 - u3))
    )
        
    return [x1_final, x2_final]


# Test with same parameters as Example 6 in feed-tank-models.ipynb
D = 5  # tank diameter [m]
A = np.pi * D ** 2 / 4  # tank cross-sectional area [m^2]
L = 5
m = 25
v_dot_in = 1.5
conc_in = 0.1
v_dot_out = 1
assert ivp_solution(0.25, L, m, v_dot_in, conc_in, v_dot_out, A) \
    == [5.0063661977236755, 24.97391173654701]

## Compare analytical solution to RK4 numerical solution

In [ ]:
dt = 0.25
tank_model_dt = MixingTankModelDT(D=D)

D = 5  # tank diameter [m]
A = np.pi * D ** 2 / 4  # tank cross-sectional area [m^2]
rng = np.random.default_rng(0)
for _ in range(10):
    L = float(rng.uniform(1, 5))
    m = float(rng.uniform(10, 30))
    v_dot_in = float(rng.uniform(0, 2))
    conc_in = float(rng.uniform(0, 1))
    v_dot_out = float(rng.uniform(0, 2))
    xkp1_anal = ivp_solution(dt, L, m, v_dot_in, conc_in, v_dot_out, A)
    xk = cas.vertcat(L, m)
    uk = cas.vertcat(v_dot_in, conc_in, v_dot_out)
    xkp1_num = np.array(tank_model_dt.F(0.0, xk, uk)).reshape(-1)
    assert np.allclose(xkp1_anal, xkp1_num)

## Discrete-Time Model Using Analytical Solution

In [ ]:
# Constants
D = 5  # tank diameter [m]
A = float(np.pi * D ** 2 / 4)  # tank cross-sectional area [m^2]

# Symbolic variables
t = cas.SX.sym("t")
L = cas.SX.sym("L")
m = cas.SX.sym("m")
v_dot_in = cas.SX.sym("v_dot_in")
conc_in = cas.SX.sym("conc_in")
v_dot_out = cas.SX.sym("v_dot_out")

dt = 1
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc_in, v_dot_out)
xkp1 = cas.vcat(
    ivp_solution(
        dt, L, m, v_dot_in, conc_in, v_dot_out, A, exp=cas.exp, log=cas.log
    )
)

# State transition function
F = cas.Function(
    "F",
    [t, xk, uk],
    [xkp1],
    ["t", "xk", "uk"],
    ["xkp1"],
)

# Output function
yk = xk
H = cas.Function(
    "H",
    [t, xk, uk],
    [yk],
    ["t", "xk", "uk"],
    ["yk"],
)

n = xk.shape[0]
nu = uk.shape[0]
ny = yk.shape[0]

tank_model_dt_anal = StateSpaceModelDT(
    F, 
    H, 
    n, 
    nu, 
    ny, 
    dt=dt,
    input_names=["v_dot_in", "conc_in", "v_dot_out"],
    state_names=["L", "m"],
    output_names=["L", "m"],
)
tank_model_dt_anal

In [ ]:
nT = 250
simulate_anal = make_n_step_simulation_function_from_model(
    tank_model_dt_anal, nT
)
simulate_anal

In [ ]:
# Simulation time
Ts = 1.0
t_eval = Ts * np.arange(nT+1)

# Input sequence
v_dot_in = 1.5
conc_in = 0.1
v_dot_out = 1
U = np.full((nT, nu), np.nan)
U[:, 0] = v_dot_in
U[:, 1] = conc_in
U[:, 2] = v_dot_out
assert U.shape == (nT, nu)

# Initial condition
L = 5
m = 25
x0 = [L, m]

assert t_eval.shape == (nT + 1, )
X_anal, Y_anal = simulate_anal(t_eval, U, x0)
assert X_anal.shape == (nT + 1, n)
assert Y_anal.shape == (nT + 1, ny)

In [ ]:
def make_tank_tsplot(X, dt, m_final=None, y_lim=([-0.5, 45.5]), title="Model States"):
    nT = X.shape[0] - 1
    t = dt * np.arange(0, nT+1)
    plt.plot(t, X)
    if m_final is not None:
        plt.hlines(m_final, t[0], t[-1], color='C1', linestyle='--')
    plt.grid()
    plt.xlabel('Time $t$')
    plt.ylabel(r'Output Variables')
    plt.ylim(y_lim)
    plt.legend(labels=['Tank level', 'Total mass', 'Mass in steady-state'], loc='best')
    plt.title(title)
    return plt.gca()


ax = make_tank_tsplot(X_anal, dt)
plt.show()